# Seleção de Atributos (variáveis)

# Métricas

### Selecionar os melhores atributos, depois treinar o modelo e aplicar as métricas

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Criando um modelo
from sklearn import linear_model

def calcula_metricas(dataset, atributos_selecionados):

    X = dataset.loc[:, atributos_selecionados ].values
    y = dataset['target'].values

    modelo = linear_model.LinearRegression() 
    modelo.fit(X,y)

    # MAE - Mean Absolute Error
    from sklearn import model_selection
    kfold = 10
    resultado = model_selection.cross_val_score(modelo, X, y, cv = kfold, 
                                                scoring = 'neg_mean_absolute_error')

    # Print do resultado
    print("MAE: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

    # MSE - Mean Squared Error
    # Definindo os valores para o número de folds
    num_folds = 10
    num_instances = len(X)
    seed = 7

    # Separando os dados em folds
    kfold = model_selection.KFold(num_folds, True, random_state = seed)

    resultado = model_selection.cross_val_score(modelo, X, y, cv = kfold, scoring = 'neg_mean_squared_error')

    # Print do resultado
    print("MSE: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

    # RMSE (Root Mean Squared Error
    from math import sqrt
    print("RMSE: %.3f " % (sqrt(abs(resultado.mean()))))

    # R2
    resultado = model_selection.cross_val_score(modelo, X, y, cv = kfold, scoring = 'r2')

    # Print do resultado
    print("R^2: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

# Seleciona os atributos
atributos_selecionados = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', # 'DIS', 
                              'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'] # 'AGE',

#calcula_metricas(dataset, atributos_selecionados)

## Carregando o Dataset Boston Houses

1. CRIM: per capita crime rate by town 
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft. 
3. INDUS: proportion of non-residential acres per town 
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) 
5. NOX: nitric oxides concentration (parts per 10 million) 
6. RM: average number of rooms per dwelling 
7. AGE: proportion of owner-occupied units built prior to 1940 
8. DIS: weighted distances to five Boston employment centres 
9. RAD: index of accessibility to radial highways 
10. TAX: full-value property-tax rate per 10,000 
11. PTRATIO: pupil-teacher ratio by town 
12. B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town 
13. LSTAT: % lower status of the population 
14. TARGET: Median value of owner-occupied homes in $1000's

## Seleção de Atributos - Utilizando RFE - Recursive Feature Elimination

Esta técnica para seleção de atributos, que recursivamente remove os atributos e constrói o modelo com os atributos remanescentes. <br> Esta técnica utiliza a acurácia do modelo para identificar os atributos que mais contribuem para prever a variável alvo. <br> Em inglês esta técnica é chamada Recursive Feature Elimination (RFE).


In [ ]:
# Gerando o dataset
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston() 
dataset = pd.DataFrame(boston.data, columns = boston.feature_names)
dataset['target'] = boston.target
dataset.head()

In [ ]:
print('Número de Atributos:', len(dataset.columns) -1)

In [ ]:
dataset.columns

# Aplicando a RFE para Seleção dos Melhores Atributos

In [ ]:
%%time 

# aplicando a RFE
from sklearn.feature_selection import RFE
from sklearn.svm import SVR


X = dataset.iloc[:,:-1].values
y = dataset['target'].values

num_atributos_relevantes = 8
estimator = SVR(kernel="linear")
selector = RFE(estimator, num_atributos_relevantes, step=1)
selector = selector.fit(X, y)

print("Num Features: ", selector.n_features_)

print(dataset.columns)
print("Selected Features: ", selector.support_)
print("Feature Ranking: ", selector.ranking_)

In [ ]:
dfatributos = pd.DataFrame( {'Atributo': dataset.columns[:-1],
                              'Importancia': selector.ranking_ })
                             
dfatributos = dfatributos.sort_values(by='Importancia', ascending=True)
dfatributos

In [ ]:
dfatributos.Atributo.values

# Comparação de Modelos

In [ ]:
print("Modelo com 13 atributos")
atributos_selecionados = ['CRIM', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT',
       'ZN', 'AGE', 'B', 'RAD', 'TAX']
calcula_metricas(dataset, atributos_selecionados)

print("\nModelo com atributos selecionados com RFE")
# Seleciona os atributos
atributos_selecionados = ['CRIM', 'INDUS', 'CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT']
       #'ZN', 'AGE', 'B', 'RAD', 'TAX']

calcula_metricas(dataset, atributos_selecionados)

## Seleção de Atributos -  Seleciona os atributos pela variância

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold()
selector.fit_transform(X)
selector.variances_

In [ ]:
dfatributos2 = pd.DataFrame( {'Atributo': dataset.columns[:-1],
                              'Importancia': selector.variances_  })
                             
dfatributos2 = dfatributos2.sort_values(by='Importancia', ascending=True).reset_index(drop=True)
dfatributos2

In [ ]:
dfatributos2.Atributo.values

In [ ]:
print("Modelo com 13 atributos")
atributos_selecionados = ['NOX', 'CHAS', 'RM', 'DIS', 'PTRATIO', 'RAD', 'INDUS', 'LSTAT',
       'CRIM', 'AGE', 'ZN', 'B', 'TAX']
calcula_metricas(dataset, atributos_selecionados)

print("\nModelo com atributos selecionados com VarianceThreshold")
# Seleciona os atributos
atributos_selecionados = ['NOX', 'CHAS', 'RM', 'DIS', 'PTRATIO', 'RAD', 'INDUS', 'LSTAT',
       'CRIM', 'AGE'] #'ZN', 'B', 'TAX']

calcula_metricas(dataset, atributos_selecionados)

## Seleção de Atributos - Utilizando Árvores de Decisão 

Bagged Decision Trees, como o algoritmo RandomForest, podem ser usados para estimar a importância de cada atributo. Esse método retorna um score para cada atributo.

Quanto maior o score, maior a importância do atributo.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(n_estimators=30, max_depth=6)
clf = clf.fit(X, y)
clf.feature_importances_           

In [ ]:
dfatributos3 = pd.DataFrame( {'Atributo': dataset.columns[:-1],
                              'Importancia': clf.feature_importances_  })
                             
dfatributos3 = dfatributos3.sort_values(by='Importancia', ascending=False)
dfatributos3

# Atividade 1-A: Criar o modelo e as métricas, comparando com o modelo original

## Seleção de Atributos - Extra Trees Classifier

In [ ]:
# Feature Importance with Extra Trees Classifier

from sklearn.ensemble import ExtraTreesRegressor

# feature extraction
model = ExtraTreesRegressor()
model.fit(X, y)
print(model.feature_importances_)

In [ ]:
dfatributos4 = pd.DataFrame( {'Atributo': dataset.columns[:-1],
                              'Importancia': model.feature_importances_ })
                             
dfatributos4 = dfatributos3.sort_values(by='Importancia', ascending=False).reset_index(drop=True)
dfatributos4

In [ ]:
# Seleciona os atributos
atributos_selecionados = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', # 'DIS', 
                              'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'] # 'AGE',

calcula_metricas(dataset, atributos_selecionados)

# Atividade 1-B: Criar o modelo e as métricas, comparando com o modelo original